In [ ]:
# API token and CA crt

$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# token from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
$caCrtFilename 
# API server URL
$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'
$apiServerUrl

cp-api-explorer-token-sbfpk
./k8sca.crt
"https://192.168.64.6:8443"


In [ ]:
# test - name of first pod in default NS
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[0] | .metadata.name'

web-1


In [ ]:
# list of namespaces
$namespaces = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[] | .metadata.name'
$namespaces

default
kube-node-lease
kube-public
kube-system


In [ ]:
# learn from masters - verbose mode of kubectl
kubectl get svc -v=8

I0331 15:11:28.672594    2205 loader.go:372] Config loaded from file:  /Users/mkoldov/.kube/config
I0331 15:11:28.673649    2205 cert_rotation.go:137] Starting client certificate rotation controller
I0331 15:11:28.678320    2205 round_trippers.go:463] GET https://192.168.64.6:8443/api/v1/namespaces/default/services?limit=500
I0331 15:11:28.678330    2205 round_trippers.go:469] Request Headers:
I0331 15:11:28.678337    2205 round_trippers.go:473]     User-Agent: kubectl/v1.23.3 (darwin/amd64) kubernetes/816c97a
I0331 15:11:28.678342    2205 round_trippers.go:473]     Accept: application/json;as=Table;v=v1;g=meta.k8s.io,application/json;as=Table;v=v1beta1;g=meta.k8s.io,application/json
I0331 15:11:28.687319    2205 round_trippers.go:574] Response Status: 200 OK in 8 milliseconds
I0331 15:11:28.687339    2205 round_trippers.go:577] Response Headers:
I0331 15:11:28.687347    2205 round_trippers.go:580]     Audit-Id: 0495e912-83e3-4374-b963-445f9a58915d
I0331 15:11:28.687354    2205 round_t

In [ ]:
# verbose - list services (expect to fail - no permissions to access SERVICES!)
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/services" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `


{
  "kind": "Status",
  "apiVersion": "v1",
  "metadata": {},
  "status": "Failure",
  "message": "services is forbidden: User \"system:serviceaccount:default:cp-api-explorer\" cannot list resource \"services\" in API group \"\" in the namespace \"default\"",
  "reason": "Forbidden",
  "details": {
    "kind": "services"
  },
  "code": 403
}


In [ ]:
# all pods per namespace with IP address
$namespaces | % {
    $namespace = $_
    /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
}

In [ ]:
# all pods per namespace with IP address
$objects = $namespaces | % {
    $namespace = $_
    $pods = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
    $ips = $pods | % { $_.ip }
    if ($ips.Length -gt 0) { 
        # Write-Host $namespace ": " $ips 
        [PSCustomObject] @{
            name = "ns-$namespace-ips";
            id = "id-ns-$namespace-ips";
            description = "IP adresses in namespace $namespace";
            ranges = $ips
        }
    }
}

$objects

In [ ]:
[PSCustomObject] @{
        "version" = "1.0";
        "description" = "Generic Data Center file example";
        "objects" = $objects
} | ConvertTo-Json -Depth 10 | jq .